In [ ]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

In [ ]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "pending_enrichment" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

import pandas as pd
pd.options.display.max_rows=999
pd.options.display.max_columns=999

In [ ]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''

    sort_columns: list # List of Long-ID, Pharmacy Code, Brand/Medication and Status Date used to sort the dataframe. Order of variables matters
    substatus_list: list # List of integrichain substatuses for current customer. Order of variables does not matter.
    pjh: str # Patient Journey Hiearchy column
    ic_status: str # Integrichain Status Column, should have been added in an early transform
    ic_substatus: str # Integrichain Sub Status Column, should have been added in an early transform
    bvpa: str # BV/PA string in Patient Journey Hierarchy column. String should be something like 'BV/PA'
    intake: str # Intake string in Patient Journey Hierarchy column. String should be something like 'INTAKE'
    fulfillment: str # Fulfillment string in Patient Journey Hierarchy column. String should be something like 'FULFILLMENT'
    pending: str # Pending string in Integrichain Status column. String should be something like 'PENDING'
    
    
    def pending_enrichment(self,df):

        df = df.sort_values(self.sort_columns,ascending=[True, True, True, True])

        groupby_cols = self.sort_columns[:-1]
        status_date = self.sort_columns[3]
        
        bvpa_df = df[(df[self.pjh] == self.bvpa)]

        intake_df = df[(df.Patient_Journey_Hierarchy.isin([self.bvpa,self.intake]))]

        fulfillment_df = df[(df.Patient_Journey_Hierarchy.isin([self.bvpa,self.fulfillment]))]

        min_bvpa = bvpa_df.groupby(groupby_cols)[status_date].min().reset_index(drop=False).rename(columns={status_date:'min_bvpa_date'})
        max_bvpa = bvpa_df.groupby(groupby_cols)[status_date].max().reset_index(drop=False).rename(columns={status_date:'max_bvpa_date'})
        max_bvpa_intake = intake_df.groupby(groupby_cols)[status_date].max().reset_index(drop=False).rename(columns={status_date:'max_bvpa_intake_date'})
        min_bvpa_fulfillment = fulfillment_df.groupby(groupby_cols)[status_date].min().reset_index(drop=False).rename(columns={status_date:'min_bvpa_fulfillment'})

        df = pd.merge(df,min_bvpa,on=groupby_cols,how='left')
        df = pd.merge(df,max_bvpa_intake,on=groupby_cols,how='left')
        df = pd.merge(df,max_bvpa,on=groupby_cols,how='left')
        df = pd.merge(df,min_bvpa_fulfillment,on=groupby_cols,how='left')

        enrich_df = df[(df[self.ic_status] == self.pending) & (df[self.ic_substatus].isin(self.substatus_list))]

        df = df[~((df[self.ic_status] == self.pending) & (df[self.ic_substatus].isin(self.substatus_list)))]

        enrich_df.loc[:,'Before_min_fulfillment'] = np.where(enrich_df.loc[:,status_date] < enrich_df.loc[:,'min_bvpa_fulfillment'],1,0)
        enrich_df.loc[:,'Before_min_BVPA'] = np.where(enrich_df.loc[:,status_date] < enrich_df.loc[:,'min_bvpa_date'],1,0)
        enrich_df.loc[:,'After_max_BVPA'] = np.where(enrich_df.loc[:,status_date] > enrich_df.loc[:,'max_bvpa_date'],1,0)
        enrich_df.loc[:,'After_max_intake'] = np.where(enrich_df.loc[:,status_date] > enrich_df.loc[:,'max_bvpa_intake_date'],1,0)

        enrich_df.loc[:,self.pjh] = (
            np.where(enrich_df.loc[:,'Before_min_fulfillment'] == 1, self.intake,
            np.where((enrich_df.loc[:,'Before_min_BVPA'] == 0) & (enrich_df.loc[:,'After_max_BVPA'] == 0),self.bvpa,
            np.where(enrich_df.loc[:,'After_max_intake'] == 1, self.fulfillment,
            enrich_df.loc[:,self.pjh])))
        )
        
        df = pd.concat([df,enrich_df],sort=False)
        
        #df = df.drop(labels=['After_max_BVPA','After_max_intake','Before_min_BVPA','Before_min_fulfillment'],axis=1)
        
        return df
    
transform = Transform()

In [ ]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

transform.sort_columns = ['msa_patient_id', 'pharm_code', 'medication', 'status_date']
transform.pjh = 'Patient_Journey_Hierarchy'
transform.ic_status = 'integrichain_status'
transform.ic_substatus = 'integrichain_sub_status'
transform.bvpa = 'BV/PA'
transform.intake = 'INTAKE'
transform.fulfillment = 'FULFILLMENT'
transform.pending = 'PENDING'
transform.substatus_list = ['NEW', 'BENEFITS', 'PA', 'PATIENT HOLD', 'PRESCRIBER HOLD', 'PATIENT CONTACT', 'INVENTORY HOLD', 'THERAPY HOLD', 'OTHER', 
                            'APPEAL', 'DELAY', 'INFORMATION', 'PATIENT RESPONSE', 'PRESCRIBER']

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

![Pending Enrichment Flowchart](assets/PendingEnrichment.png)

Replaces ambiguous substatus with info based on sub-statuses surrounding the ambiguous field

### Transformation

In [ ]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
# run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[1]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
df = input_contract.fetch(filters=run_filter)

import numpy as np
import pandas as pd
pd.options.display.max_columns=999

df.status_date = df.status_date.str[:8].astype(str)
df.ref_date = df.ref_date.str[:8].astype(str)

df.status_date = pd.to_datetime(df.status_date, infer_datetime_format=True, errors='coerce')
df.ref_date = pd.to_datetime(df.ref_date, infer_datetime_format=True, errors='coerce')

In [ ]:
import os
os.chdir('{}'.format(os.path.expanduser('~')))
status_config = pd.read_csv('status_mapping.csv')

status_config.loc[:,'statusCode'] = status_config.statusCode.str.upper()
status_config.loc[:,'subStatus'] = status_config.subStatus.str.upper()
status_config.loc[:,'integrichain_sub_status'] = status_config.integrichain_sub_status.str.upper()
status_config.loc[:,'integrichain_status'] = status_config.integrichain_status.str.upper()
status_config.loc[:,'Patient_Journey_Hierarchy'] = status_config.Patient_Journey_Hierarchy.str.upper()

status_config = status_config.rename(columns={'statusCode':'status_code','subStatus':'sub_status'})

df.sub_status = df.sub_status.str.replace('PRESCRIBERHOLD','PRESCRIBER HOLD')

df = pd.merge(df,status_config,on=['status_code','sub_status'])

df = df[['rec_date', 'pharm_code', 'pharm_npi', 'transtype', 'pharm_transaction_id', 'trans_seq', 'ref_source', 'ref_date', 'program_id', 'pharmacy_id', 'pat_last_name', 'pat_first_name', 'pat_dob', 'pat_gender', 
         'pat_addr1', 'pat_addr2', 'pat_city', 'pat_state', 'pat_zip', 'dx1_code', 'dx2_code', 'status_date', 'status_code', 'sub_status', 'integrichain_status','integrichain_sub_status', 'Patient_Journey_Hierarchy', 
         'pres_last_name', 'pres_first_name', 'pres_addr1', 'pres_addr2', 'pres_city', 'pres_state', 'pres_zip', 'pres_phone', 'pres_npi', 'pres_dea', 'facility_name', 'rxdate', 'rxnumber', 'rxrefills', 'rxfill', 
         'refill_remaining', 'prev_disp', 'rx_ndc_number', 'medication', 'quantity', 'day_supply', 'ship_date', 'ship_carrier', 'shiptracking_num', 'ship_location', 'ship_address', 'ship_city', 'ship_state', 'ship_zip', 
         'has_medical', 'primary_coverage_type', 'primary_payer_name', 'primary_payer_type', 'secondary_coverage_type', 'secondary_payer_name', 'secondary_payer_type', 'plan_paid_amt', 'pat_copay', 'copay_assist_amount', 
         'oth_payer_amt', 'xfer_pharmname', 'msa_patient_id', 'msa_patient_bmap', '__metadata_run_timestamp', '__metadata_app_version', '__metadata_output_contract', '__metadata_transform_timestamp', '__metadata_run_id']]

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe
final_dataframe = transform.pending_enrichment(df)

In [ ]:
import unittest

def shape_status(final_dataframe,df):
    return df.shape[0] == final_dataframe.shape[0]

def substatus_cleaned(final_dataframe,sub_col):
    return final_dataframe[final_dataframe[sub_col].isna()]

class TestNotebook(unittest.TestCase):
    
    def test_shape_status(self):
        self.assertEqual(shape_status(final_dataframe,df),True)
        
    def test_substatus_cleaned(self):
        substatus_cleaned(final_dataframe,'Patient_Journey_Hierarchy')
    
unittest.main(argv=[''],verbosity=2,exit=False)

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()